# Import Packages

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

#Own packages
import louvainFunctions_v3 as lF
import markergeneFunctions as mgF
import plotumapFunctions as puF
import evaluation as ev
import gnbFunctions as gnb

# Pre-process data and dimensionality reduction

Log transform and normalization, dimensionality reduction using PCA


In [ ]:
data = np.loadtxt("../output/data_loaded.csv", dtype=float, delimiter=",")
data[data == 0.0] = 0.0000001
print("Data is loaded. Now taking log-transform. ")

data = log_transform(data)
print("Data is transformed. Now scaling.")

data = scale(data)
print("Data is mean-centred. Now performing PCA.")

variances, reduced_data = perform_pca(data)
print("PCA successful! Now saving data.")

sample = reduced_data[:500, :]
np.savetxt("../output/reduced_sample_20_PCs.csv", sample, delimiter=",", fmt="%.2f")

np.savetxt("../output/reduced_data_20_PCs.csv", reduced_data, delimiter=",", fmt="%.2f")
print("Data saved! Now plotting.")

# Load Data

In [ ]:
dirpath = None
if dirpath is None:
    raise Exception(r'MAKE DIRPATH YOUR DIRECTORY SUCH THAT THE FOLDER "data" IS IN THE DIRECTORY')
os.chdir(dirpath)

In [ ]:
data = np.loadtxt(os.path.join(r'.\data', 'reduced_sample_20_PCs.csv'), delimiter=',')
print(data.shape)

In [ ]:
#import larger dataset
large_data = np.loadtxt(os.path.join(r'.\data', 'reduced_data_20_PCs.csv'), delimiter=',')
print(large_data.shape)

# Test against GMMs from package

In [ ]:
from sklearn.mixture import GaussianMixture

membership_arr_gmm = GaussianMixture(n_components=23, random_state=0, n_init = 20).fit_predict(large_data)
save_dir = os.path.join(os.getcwd(), 'output')
save_path = os.path.join(save_dir, 'gmm_clusters.txt')
np.savetxt(save_path, membership_arr_gmm[np.newaxis, :], fmt = '%d')
np.save('membership_arr_gmm_29.npy', membership_arr_gmm)

# Run Louvain
Will create an array of labels with dimensions (# of samples,)

In [ ]:
membership_arr_louvain = lF.louvain_clustering(large_data, graph_style='kNN', k=20,)
np.save('./output/membership_arr_louvain_large_k20_knn.npy', membership_arr_louvain)

# Test against Louvain from package

import networkx as nx
import networkx.algorithms.community as nx_comm
from louvainFunctions import create_kNN_graph

edge_list, edge_weights = lF.create_kNN_graph(data, 5)
G=nx.Graph()
G.add_edges_from(edge_list)

#first compute the best partition
partition = nx_comm.louvain_communities(G)

membership_arr_package = np.zeros((len(data)))
for i in range(len(partition)):
    for n in partition[i]:
        membership_arr_package[n] = i

# Plot UMAP

In [ ]:
membership_arr_louvain = np.load('membership_arr_louvain_large_k5_t10000_knn.npy')
membership_arr_gmm = np.load('membership_arr_gmm_23.npy')

In [ ]:
puF.plot_umap(large_data, [membership_arr_louvain, membership_arr_gmm, ], ['Louvain', 'GMM'])

# Get Marker Genes and plot violin plots

Will save a dictonary of marker genes for each cluster and plot violin plots for top 3 genes for each cluster. Requires original data before dimension reduction. Have not tested on this dataset.

In [ ]:
# membership_arr_louvain = np.load('membership_arr_louvain_large_k5_t10000_knn.npy')

In [ ]:
data_file = os.path.join(r'./data', 'data_mtg_loaded.hdf5')
membership_arr_louvain = mgF.collapse_small_clusters(membership_arr_louvain, num_max=23)
clusters = np.unique(membership_arr_louvain)
median_arr, mean_arr = mgF.get_gene_stats_by_cluster(data_file, membership_arr_louvain.reshape(-1), clusters)
mgF.filter_genes_by_median(median_arr, mean_arr)
save_dir = './output'
mgF.plot_marker_genes(data_file, clusters, membership_arr_louvain.reshape(-1), save_dir = save_dir)

# Evaluate performance

In [ ]:
louvain_centres = ev.calculate_cluster_centroids(membership_arr_louvain, large_data)
louvain_di = ev.dunn_index(membership_arr_louvain, large_data, louvain_centres)
louvain_sc = ev.silhouette_coefficient(membership_arr_louvain, large_data, louvain_centres)

gmm_centres = ev.calculate_cluster_centroids(membership_arr_gmm, large_data)
gmm_di = ev.dunn_index(membership_arr_gmm, large_data, gmm_centres)
gmm_sc = ev.silhouette_coefficient(membership_arr_gmm, large_data, gmm_centres)

di = np.asarray([louvain_di, gmm_di])
sc = [louvain_sc, gmm_sc]

ev.plot_evaluation_metric(di, ['Louvain','GMM'], "Dunn Index", output_folder = './output')
ev.plot_evaluation_metric(
        sc, ['Louvain', 'GMM'], "Silhouette Coefficient", output_folder = './output')

# Gaussian Naive Bayes

In [ ]:
#Run GNB on Louvain labels
X = data
y = np.array([int(n) for n in membership_arr_louvain])
features = [x for x in range(X.shape[1])]
classes = np.unique(y)

X_train, X_test, y_train, y_test = gnb.train_test_split(X, y, test_size = 0.3, random_state = 0, stratify=y)

priors_data = gnb.calculate_class_priors(y_train)
param = gnb.calculate_param(X_train, y_train)
pred_pca = np.array(gnb.predict(X_test, features, classes, priors_data, param))
print("GNB accuracy on 500 points:", gnb.accuracy_score(pred_pca, y_test))

In [ ]:
new_labels = mgF.collapse_small_clusters(membership_arr_gmm, num_max=17)
new_labels = new_labels[new_labels != np.max(new_labels)]

cl, counts = np.unique(new_labels, return_counts=True)
plt.figure()
plt.bar(cl, counts)
plt.xlabel('Class')
plt.ylabel('Counts')
plt.title('Class distribution for gmm')
plt.show()

In [ ]:
#predict on large data.
import dataFunctions as dF
import importlib
importlib.reload(ev)
importlib.reload(mgF)
importlib.reload(dF)

X_large = large_data

new_labels = mgF.collapse_small_clusters(membership_arr_gmm, num_max=17)
print(np.unique(new_labels))
X_large = large_data[new_labels != np.max(new_labels)]
# X_large = large_data

new_labels = new_labels[new_labels != np.max(new_labels)]

y = np.array([int(n) for n in new_labels])
features = [x for x in range(X_large.shape[1])]
classes = np.unique(y)
print(classes)


# X_train, X_test, y_train, y_test = gnb.train_test_split(X_large, y, test_size = 0.5, random_state=42, stratify=y)
X_train, y_train, X_test, y_test = dF.split_data(X_large, y)


priors_data = gnb.calculate_class_priors(y_train)
param = gnb.calculate_param(X_train, y_train)
pred_pca = np.array(gnb.predict(X_test, features, classes, priors_data, param))
print("GNB accuracy on 16155 points:", gnb.accuracy_score(pred_pca, y_test))

ev.confusion_matrix(y_test, pred_pca)